In [ ]:
%load_ext autoreload
%autoreload 2
from new_aeb_gplvm import *
from utils.data_generator import DataGenerator
from utils.myutils import Utils

datagenerator = DataGenerator()  # data generator
utils = Utils()  # utils function
from tqdm import trange
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import random

random.seed(42)

In [ ]:
# dataset and model list / dict
dataset_list = [
    "01_ALOI",
    "02_annthyroid",
    "03_backdoor",
    "04_breastw",
    "05_campaign",
    "06_cardio",
    "07_Cardiotocography",
    "08_celeba",
    "09_census",
    "99_linear",
    "99_circles",
    "99_moons",
    "99_clusters",
    "99_oilflow",
]
dataset = dataset_list[2]
datagenerator.dataset = dataset

data = datagenerator.generator(
    la=0.999,
    realistic_synthetic_mode=None,
    stdscale=True,
    minmax=False,
    noise_type=None,
)

Y_train = torch.tensor(data["X_train"], dtype=torch.float32)
Y_test = torch.tensor(data["X_test"], dtype=torch.float32)
lb_train = torch.tensor(data["y_train"], dtype=torch.float32)
lb_test = torch.tensor(data["y_test"], dtype=torch.float32)
idx_n = np.where(lb_train == 0)[0]
Y_train = Y_train[idx_n]
lb_train = lb_train[idx_n]

In [ ]:
n_train = len(Y_train)
data_dim = Y_train.shape[1]
latent_dim = 15
n_inducing = 50
n_epochs = 1000
nn_layers = 15, 15
lr = 0.01
batch_size = 128

In [20]:
gplvm = AD_GPLVM(latent_dim, n_inducing, n_epochs, nn_layers, lr, batch_size)

In [22]:
gplvm.fit(Y_train)

Setando Kernel RBF Padrão


Loss: 318.07, iter no: 0: 100%|██████████| 1/1 [00:00<00:00,  5.43it/s]

torch.Size([6836, 15])
MultivariateNormal(loc: torch.Size([196, 128]))


In [ ]:
X_prior_mean = torch.zeros(n_train, latent_dim)
X_prior_covar = torch.eye(X_prior_mean.shape[1])
prior_x = MultivariateNormalPrior(X_prior_mean, X_prior_covar)

encoder = NN_Encoder(n_train, latent_dim, prior_x, data_dim, nn_layers)

model = GP_Decoder(
    n_train,
    data_dim,
    latent_dim,
    n_inducing,
    encoder,
    nn_layers,
)

likelihood = GaussianLikelihood()

optimizer = torch.optim.Adam(
    [
        {"params": model.parameters()},
        {"params": likelihood.parameters()},
    ],
    lr,
)

loss_list = []

elbo = VariationalELBO(likelihood, model, num_data=len(Y_train))
model.train()

iterator = trange(1, leave=True)

for i in iterator:
    batch_index = model._get_batch_idx(batch_size)
    optimizer.zero_grad()
    sample = model.sample_latent_variable(Y_train)
    sample_batch = sample[batch_index]
    output_batch = model(sample_batch)
    print(sample.shape)
    print(output_batch)
    loss = -elbo(output_batch, Y_train[batch_index].T).sum()
    loss_list.append(loss.item())
    iterator.set_description(
        "Loss: " + str(float(np.round(loss.item(), 2))) + ", iter no: " + str(i)
    )
    loss.backward()
    optimizer.step()

In [ ]:
with torch.no_grad():
    model.eval()
    likelihood.eval()


def create_dist_qx(model, batch_target):
    mu = model.predict_latent(batch_target)[0]
    sigma = model.predict_latent(batch_target)[1]
    local_q_x = MultivariateNormal(mu, sigma)
    return mu, sigma, local_q_x


def create_dist_px(
    batch_target,
    mu,
):
    local_p_x_mean = torch.zeros(batch_target.shape[0], mu.shape[1])
    local_p_x_covar = torch.eye(mu.shape[1])
    local_p_x = MultivariateNormalPrior(local_p_x_mean, local_p_x_covar)
    return local_p_x


def kl_divergence_variational(target):
    ll_shape = torch.zeros_like(target.T)
    klu = model.variational_strategy.kl_divergence().div(batch_size)
    klu_expanded = ll_shape.T.add_(klu).sum(-1).T.div((n_train))
    return klu_expanded


def predict_score(X_test):
    n_test = len(X_test)
    mu, sigma, local_q_x = create_dist_qx(model, X_test)
    local_p_x = create_dist_px(X_test, mu)
    X_pred = model(model.sample_latent_variable(X_test))
    log_likelihood = likelihood.expected_log_prob(X_test.T, X_pred).sum(0).div(n_test)
    kl_x = kl_divergence(local_q_x, local_p_x).div(n_test)
    kl_u = kl_divergence_variational(X_test)
    score = -(log_likelihood - kl_u - kl_x).detach().numpy()
    score = MinMaxScaler().fit_transform(np.reshape(score, (-1, 1)))
    return score


utils.metric(y_true=lb_test, y_score=predict_score(Y_test))

In [ ]:
plt.plot(loss_list)